In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [14]:
# populate database
import os
import shutil
import sqlite3

import pandas as pd
import requests

db_url = "https://storage.googleapis.com/benchmarks-artifacts/travel-db/travel2.sqlite"
local_file_path = 'db/travel2.sqlite'
backup_file_path = "db/travel2.backup.sqlite"
overwrite = False

if overwrite or not os.path.exists(local_file_path):
  response = requests.get(db_url)
  response.raise_for_status()
  with open(local_file_path, 'wb') as f:
    f.write(response.content)
    
  shutil.copy(local_file_path, backup_file_path)

In [58]:
from sqlite3 import connect


# Convert the flights to present time for our tutorial
def update_dates(file):
    shutil.copy(backup_file_path, file)
    conn = sqlite3.connect(file)
    cursor = conn.cursor()

    tables = pd.read_sql(
        "SELECT name FROM sqlite_master WHERE type='table';", conn
    ).name.tolist()
    tdf = {}
    for t in tables:
        tdf[t] = pd.read_sql(f"SELECT * from {t}", conn)

    example_time = pd.to_datetime(
        tdf["flights"]["actual_departure"].replace("\\N", pd.NaT)
    ).max()
    current_time = pd.to_datetime("now").tz_localize(example_time.tz)
    time_diff = current_time - example_time

    tdf["bookings"]["book_date"] = (
        pd.to_datetime(tdf["bookings"]["book_date"].replace("\\N", pd.NaT), utc=True)
        + time_diff
    )

    datetime_columns = [
        "scheduled_departure",
        "scheduled_arrival",
        "actual_departure",
        "actual_arrival",
    ]
    for column in datetime_columns:
        tdf["flights"][column] = (
            pd.to_datetime(tdf["flights"][column].replace("\\N", pd.NaT)) + time_diff
        )

    for table_name, df in tdf.items():
        df.to_sql(table_name, conn, if_exists="replace", index=False)
    del df
    del tdf
    conn.commit()
    conn.close()

    return file

update_dates(local_file_path)

'db/travel2.sqlite'

In [57]:
conn = sqlite3.connect(local_file_path)
cursor = conn.cursor()

pd.read_sql("SELECT * from flights WHERE status IS 'Scheduled' limit 20;", conn)

,flight_id,flight_no,scheduled_departure,scheduled_arrival,departure_airport,arrival_airport,status,aircraft_code,actual_departure,actual_arrival
0,1185,QR0051,2024-05-26 03:59:03.561731-04:00,2024-05-26 09:04:03.561731-04:00,BSL,BKK,Scheduled,319,None,None
1,3979,MU0066,2024-05-10 08:59:03.561731-04:00,2024-05-10 11:44:03.561731-04:00,SHA,CUN,Scheduled,CR2,None,None
2,4739,QF0126,2024-05-21 06:39:03.561731-04:00,2024-05-21 08:24:03.561731-04:00,SHA,AMS,Scheduled,763,None,None
3,5502,LX0136,2024-05-28 03:59:03.561731-04:00,2024-05-28 05:29:03.561731-04:00,OSL,PRG,Scheduled,763,None,None
4,6938,IB0075,2024-05-20 06:34:03.561731-04:00,2024-05-20 07:29:03.561731-04:00,OSL,RGN,Scheduled,SU9,None,None
5,7784,QF0028,2024-05-26 09:09:03.561731-04:00,2024-05-26 11:39:03.561731-04:00,OSL,EDI,Scheduled,CR2,None,None
6,9478,KE0134,2024-05-13 03:09:03.561731-04:00,2024-05-13 05:44:03.561731-04:00,HAM,MNL,Scheduled,CR2,None,None
7,11085,AC0107,2024-05-09 09:14:03.561731-04:00,2024-05-09 10:19:03.561731-04:00,PEK,NBO,Scheduled,733,None,None
8,11847,CA0065,2024-05-28 04:24:03.561731-04:00,2024-05-28 09:04:03.561731-04:00,GVA,DEN,Scheduled,319,None,None
9,12012,IB0029,2024-05-11 10:14:03.561731-04:00,2024-05-11 11:09:03.561731-04:00,GVA,HNL,Scheduled,CR2,None,None
